In [5]:
import sys
import os

# Add the absolute path one level up
sys.path.append(os.path.abspath(".."))
import neurongraph as ng

In [6]:
g = ng.NeuronGraph("../test/neuron.swc")

In [5]:
g.numberOfNodes()

505

In [6]:
nodes = g.getNodes()

In [17]:
for ind , node in nodes.items():
    print(f"Node {ind}: ({node.x}, {node.y}, {node.z}), RADIUS: {node.radius}, PID")

Node 1: (0.0, 0.0, 0.0)
Node 2: (10.53, 1.47, -0.88)
Node 3: (-10.52, -1.47, 0.88)
Node 4: (4.0, -10.03, -0.45)
Node 5: (7.61, -14.97, -0.67)
Node 6: (15.88, -17.33, 11.16)
Node 7: (18.47, -19.27, 11.02)
Node 8: (28.28, -26.69, 23.49)
Node 9: (27.83, -36.68, 22.88)
Node 10: (29.8, -49.62, 22.93)
Node 11: (36.95, -64.21, 34.51)
Node 12: (41.41, -75.0, 34.43)
Node 13: (44.21, -81.16, 34.4)
Node 14: (44.94, -88.1, 34.5)
Node 15: (50.27, -97.75, 35.78)
Node 16: (54.95, -103.75, 38.41)
Node 17: (59.88, -119.98, 39.27)
Node 18: (66.93, -131.85, 40.29)
Node 19: (70.42, -144.56, 40.41)
Node 20: (74.9, -159.74, 34.82)
Node 21: (79.89, -172.1, 39.76)
Node 22: (88.99, -182.12, 39.18)
Node 23: (94.43, -195.33, 38.95)
Node 24: (104.24, -214.18, 38.56)
Node 25: (36.81, -27.76, 31.18)
Node 26: (40.86, -31.95, 31.03)
Node 27: (50.82, -34.23, 38.17)
Node 28: (58.49, -37.11, 38.9)
Node 29: (66.81, -41.16, 48.26)
Node 30: (71.09, -42.47, 48.04)
Node 31: (75.19, -47.57, 51.44)
Node 32: (81.87, -52.16, 51.

In [27]:
from pythreejs import *
import numpy as np
from IPython.display import display

# Load graph
graph = ng.NeuronGraph()
graph.readFromFile("../output/test/splitSWC.swc")
nodes = graph.getNodes()  # dict[int, SWCNode]

# Extract coordinates
node_positions = {nid: (n.x, n.y, n.z) for nid, n in nodes.items()}

# Build edge list from pid structure
edge_list = [(n.id, n.pid) for n in nodes.values() if n.pid != -1]

# Helper: create cylinder between 2 points
def create_cylinder(p1, p2, radius=0.02, segments=8, color='gray'):
    p1 = np.array(p1)
    p2 = np.array(p2)
    direction = p2 - p1
    length = np.linalg.norm(direction)
    if length == 0:
        return None

        
    cyl = Mesh(
        geometry=CylinderGeometry(radiusTop=radius, radiusBottom=radius, height=length, radialSegments=segments),
        material=MeshLambertMaterial(color=color)
    )

    midpoint = (p1 + p2) / 2
    cyl.position = tuple(midpoint)

    y_axis = np.array([0, 1, 0])
    direction_norm = direction / length
    axis = np.cross(y_axis, direction_norm)
    angle = np.arccos(np.clip(np.dot(y_axis, direction_norm), -1.0, 1.0))

    if np.linalg.norm(axis) > 1e-6:
        axis = axis / np.linalg.norm(axis)
        s = np.sin(angle / 2)
        qx, qy, qz = axis * s
        qw = np.cos(angle / 2)
        cyl.quaternion = (qx, qy, qz, qw)
    elif np.dot(y_axis, direction_norm) < 0:
        cyl.quaternion = (1, 0, 0, 0)

    return cyl

# Make nodes as spheres
spheres = [
    Mesh(
        SphereGeometry(radius=0.03),
        MeshLambertMaterial(color='blue'),
        position=node_positions[nid]
    ) for nid in nodes
]

# Make cylinders
cylinders = [
    create_cylinder(node_positions[a], node_positions[b],nodes[a].radius)
    for a, b in edge_list
]
cylinders = [c for c in cylinders if c is not None]

# Build scene
scene = Scene(
    children=spheres + cylinders + [
        AmbientLight(intensity=0.4),
        DirectionalLight(position=[3, 3, 3], intensity=0.6)
    ],
    background='black'  # <-- correct place to set background
)

camera = PerspectiveCamera(position=[3, 3, 3], fov=45)
controls = OrbitControls(controlling=camera)
#renderer = Renderer(scene=scene, camera=camera, controls=[controls], width=900, height=700)
renderer = Renderer(
    scene=scene,
    camera=camera,
    controls=[controls],
    width=900,
    height=700,
    background='black'  # <-- dark background
)
display(renderer)


Renderer(camera=PerspectiveCamera(fov=45.0, position=(3.0, 3.0, 3.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.…